#CafChem tools running DFT and SAPT calculations using Psi4

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChem/blob/main/notebooks/Psi4_CafChem.ipynb)

- currently needs to have the fallback runtime set uder Tools --> Command palette

## This notebook allows you to:
- Perform energy, geometry optimization and SAPT calculations on molecules.
- start from a SMILES string or an XYZ file
- SAPT calculations require an XYZ file with both molecules in it.
- for available DFT methods, see the [functional list](https://psicode.org/psi4manual/master/dft_byfunctional.html)
- for available basis sets, see the [basis set list](https://psicode.org/psi4manual/master/basissets_tables.html)

## Requirements:
- This notebook will install conda, then restart.
- Then install psi4 and rdkit
- It will pull the CafChem tools from Github.
- It will install all needed libraries.

## Install Psi4 and Set-up

### Install conda, psi4 and rdkit

In [1]:
!python --version

Python 3.11.13


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
!mamba install psi4


Looking for: ['psi4']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   5%
conda-forge/noarch     1%[+] 0.3s
conda-forge/linux-64  13%
conda-forge/noarch     1%[+] 0.4s
conda-forge/linux-64  21%
conda-forge/noarch     9%[+] 0.5s
conda-forge/linux-64  27%
conda-forge/noarch    15%[+] 0.6s
conda-forge/linux-64  31%
conda-forge/noarch    32%[+] 0.7s
conda-forge/linux-64  40%
conda-forge/noarch    50%[+] 0.8s
conda-forge/linux-64  49%
conda-forge/noarch    69%[+] 0.9s
conda-forge/linux-64  56%
conda-forge/noarch    85%[+] 1.0s
conda-forge/linux-64  61%
conda-forge/noarch    95%[+] 1.1s
conda-forge/linux-64  63%
conda-forge/noarch   100%[+] 1.2s
conda-forge/linux-64  63%
conda-forge/noarch   100%[+] 1.3s
conda-forge/linux-64  63%
conda-forge/noarch   100%conda-forge/noarch                                
[+] 1.4s
conda-forge/linux-64  66%[+] 1.5s
conda-forge/linux-64  74%[+] 1.6s
conda-forge/linux-64  78%[+] 1.7s
conda-forge/linux-64  86%

In [3]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 161.3 MB/s eta 0:00:00


### Pull CafChem from Github and import libraries

In [6]:
!git clone https://github.com/MauricioCafiero/CafChem.git

Cloning into 'CafChem'...
remote: Enumerating objects: 656, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 656 (delta 160), reused 45 (delta 45), pack-reused 423 (from 1)
Receiving objects: 100% (656/656), 41.84 MiB | 37.26 MiB/s, done.
Resolving deltas: 100% (378/378), done.


In [7]:
import psi4
import numpy

import CafChem.CafChemPsi4 as ccp4

## Calculations with Psi4

### Check number of cores, set cores and memory

In [8]:
import os
cpuCount = os.cpu_count()
print(cpuCount)

8


In [9]:
cores = cpuCount
p4_mem = 16

### Energy from a SMILES string

In [10]:
p4_input = ccp4.smiles_to_psi4('CCO')

print("starting psi4 energy calculation")
new_calc = ccp4.psi4_calc(p4_mem, cores, "3-21G", "B3LYP")
energy, kcals = new_calc.calc_energy(p4_input)

print("Calculation done!")
print(f"Energy = {energy} ha.")

starting psi4 energy calculation
Calculation done!
Energy = -154.17906148224927 ha.


### Optimization from a SMILES string
- saves optimized structure to an XYZ file

In [11]:
p4_input = ccp4.smiles_to_psi4('CCO')

print("starting psi4 geometry optimization calculation")
new_calc = ccp4.psi4_calc(p4_mem, cores, "3-21G", "B3LYP")
energy, kcals, new_molecule = new_calc.optimize(p4_input)

ccp4.psi4_to_XYZ(new_molecule, "newmol.XYZ")

print("Calculation done!")
print(f"Energy = {energy} ha.")

starting psi4 geometry optimization calculation


Some dependencies such as QCElemental have not yet finished migration to pydantic v2. If issues are encountered please downgrade pydantic or upgrade QCElemental as appropriate
	Previous geometry is closer to target in internal coordinates, so using that one.

	Best geometry has RMS(Delta(q)) = 5.11e-09



Optimizer: Optimization complete!
Calculation done!
Energy = -154.182122580517 ha.


### Energy from an XYZ file

In [12]:
p4_input = ccp4.XYZ_to_psi4('newmol.XYZ')

print("starting psi4 energy calculation")
new_calc = ccp4.psi4_calc(p4_mem, cores, "3-21G", "B3LYP")
energy, kcals = new_calc.calc_energy(p4_input)

print("Calculation done!")
print(f"Energy = {energy} ha.")

starting psi4 energy calculation
Calculation done!
Energy = -154.18212258031323 ha.


### SAPT calculation from an XYZ file
- File must contain 2 molecules
- you must provide the number of atoms in the first molecule

In [13]:
p4_input = ccp4.XYZ_to_dimer('CafChem/data/dimer.xyz',15, -1, 1, 0, 1)

print("starting psi4 calculations")
new_calc = ccp4.psi4_calc(p4_mem,cores, "3-21G", "B3LYP")
energy, kcals = new_calc.sapt(p4_input)

print("Calculation done!")

starting psi4 calculations
Printing SAPT energies:
Electrostatic:            -0.067936 ha     103.34%
Exchange:                  0.046330 ha     -70.48%
Dispersion:               -0.009225 ha      14.03%
Induction:                -0.034907 ha      53.10%
Total SAPT:               -0.065738 ha
Calculation done!
